In [ ]:
import ee
import eeUtil
import geemap
from pathlib import PurePosixPath as Path

In [ ]:
# Initialize Earth Engine
PROJECT = 'ee-fgassert'


import google.auth
from google.colab import auth
auth.authenticate_user()
credentials, project_id = google.auth.default()
ee.Initialize(credentials=credentials, project=PROJECT)

In [ ]:
WORLD_GEOM = ee.Geometry.Polygon(
    coords=[[[-180, -90],[-180, 90],[180, 90],[180, -90],[-180, -90]]],
    proj='epsg:4326',
    geodesic=False
)
SCALE = 100
PROJECTION = ee.Projection('EPSG:4326')
WORKING_FOLDER = 'projects/earthengine-legacy/assets/users/fgassert/deforestation_working'
PALETTE = ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']


In [ ]:
sbtn_all_classes = ee.Image("projects/wri-datalab/SBTN/natLands_beta/naturalLands_allClasses_20230516")
layers_to_downsample = {
  'tree_loss_100m':ee.Image("UMD/hansen/global_forest_change_2022_v1_10").select(1),
  'disturbance_100m':ee.Image("projects/glad/GLCLU2020/Forest_type").eq(3),
  'gain_100m':ee.Image("projects/glad/GLCLU2020/Forest_type").eq(4),
  'SBTN_natural_lands_100m':sbtn_all_classes.lte(11),
  'SBTN_nonnatural_excl_builtwater_100m':sbtn_all_classes.gte(12).And(sbtn_all_classes.neq(13)).And(sbtn_all_classes.neq(16))
}
layers_to_export = {
  "tree_loss_year_100m": (
      ee.Image("UMD/hansen/global_forest_change_2022_v1_10")
      .select(3)
      .selfMask()
      .reduceResolution('min', False)
      .unmask()
  ),
  "burned_area": (
    ee.ImageCollection("ESA/CCI/FireCCI/5_1")
    .filterDate('2000-01-01', '2022-12-31')
    .select(1)
    .max()
    .unmask()
    .gt(50)
    .reproject(ee.ImageCollection("ESA/CCI/FireCCI/5_1").first().select(1).projection())
  ),
}

carbon = ee.Image("users/fgassert/Vulnerable_C_Total_2010")
esa_cci = ee.Image("users/fgassert/ESACCI_LC_300m_P1Y_2010_v207")
tree_cover_2000 = ee.Image("UMD/hansen/global_forest_change_2022_v1_10").select(1).unmask()
cci_tree_cover_2010 = esa_cci.gte(40).And(esa_cci.lt(100)).Or(esa_cci.eq(160)).Or(esa_cci.eq(170))
tree2000_not_2010 = tree_cover_2000.And(cci_tree_cover_2010.Not())
carbon_kernel_mean = carbon.multiply(cci_tree_cover_2010).selfMask().reduceNeighborhood('mean', ee.Kernel.square(10000, 'meters'), 'mask', False, 'boxcar')
carbon_filled_2000 = carbon.unmask().where(tree2000_not_2010, carbon_kernel_mean).selfMask()

layers_to_export['Vulnerable_C_Filled_2000'] = carbon_filled_2000

layers_to_export.update({
  k:im.unmask().reduceResolution('mean', False)
  for k,im in layers_to_downsample.items()
})

layers = {}
#eeUtil.createFolder(WORKING_FOLDER)
for key, image in layers_to_export.items():
  image = image.rename(key)
  asset_id = str(Path(WORKING_FOLDER, key))
  layers[key] = eeUtil.findOrSaveImage(image, asset_id, region=WORLD_GEOM)

In [ ]:

# Can test different kernel sizes..
RADII = [500, 3000, 10000, 25000, 50000]
SHOW_RADII = 3 #which radius to show
FILTER_LOW_DEFORESTATION = 0.01
SCALE = 1000
HANSEN_START_YEAR = 0

cropland_2019 = ee.ImageCollection("users/potapovpeter/Global_cropland_2019").mosaic()
cropland_2003 = ee.ImageCollection("users/potapovpeter/Global_cropland_2003").mosaic()
unavailable_land = sbtn_all_classes.neq([4,13,16]).reduce('anyNonZero') # water or urban

mask = ee.Image("UMD/hansen/global_forest_change_2022_v1_10").select('datamask')
natural = layers['SBTN_natural_lands_100m']
non_nat = layers['SBTN_nonnatural_excl_builtwater_100m']
tree_loss_all = layers['tree_loss_100m'].multiply(
  layers['tree_loss_year_100m'].unmask().gt(HANSEN_START_YEAR)).updateMask(unavailable_land)
intact_forests = ee.Image("users/potapovpeter/IFL_2000").unmask()
primary_tropical = ee.ImageCollection("UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1").mosaic().unmask()
burned_area = layers["burned_area"].unmask()
forest_disturbance = layers['disturbance_100m'].unmask()
carbon = layers['Vulnerable_C_Filled_2000']

primary_forest = intact_forests.Or(primary_tropical)
loss_ok = tree_loss_all.multiply(non_nat.And(primary_forest.Not()))
burned_area_ok = intact_forests.And(primary_tropical.Not()).And(burned_area)
forest_disturbance_ok = forest_disturbance.multiply(primary_forest.Not())

loss_excl_nonnat = tree_loss_all.multiply(natural.add(primary_forest).min(1))
loss_excl_nonnat_nonpridist = loss_excl_nonnat.multiply(ee.Image(1).subtract(forest_disturbance_ok))
loss_excl_nonnat_nonpridist_nontropfire = loss_excl_nonnat_nonpridist.multiply(burned_area_ok.Not()).updateMask(unavailable_land)

deforest_carbon = loss_excl_nonnat_nonpridist_nontropfire.multiply(carbon).unmask().updateMask(unavailable_land)

def convolveRadiiToBands(image, bufferSizes, optimize=True):
    bands = []
    for buffer in bufferSizes:
        bandNames = image.bandNames().map(lambda n: ee.String(n).cat(f'_{buffer}m'))
        if optimize:
            kernel = ee.Kernel.square(buffer, "meters")
            bands.append(image.reduceNeighborhood(
                "mean", kernel, "mask", False, "boxcar").rename(bandNames))
        else:
            kernel = ee.Kernel.circle(buffer, "meters")
            bands.append(image.reduceNeighborhood(
                "mean", kernel, "mask", False).rename(bandNames))
    return image.addBands(bands)

tree_loss_all = tree_loss_all.updateMask(unavailable_land)
loss_excl_nonnat_nonpridist_nontropfire = loss_excl_nonnat_nonpridist_nontropfire.updateMask(unavailable_land)
non_nat = non_nat.updateMask(unavailable_land)
deforest_carbon = deforest_carbon.updateMask(unavailable_land)

forest_loss_kernel = convolveRadiiToBands(tree_loss_all.rename('forestloss'), RADII, False)
deforest_kernel = convolveRadiiToBands(loss_excl_nonnat_nonpridist_nontropfire.rename('forestloss'), RADII, False)
non_nat_kernel = convolveRadiiToBands(non_nat.rename('nonnatural'), RADII, False)
deforest_carbon_kernel = convolveRadiiToBands(deforest_carbon.rename('carbon'), RADII, False)

# filter low deforestation
forest_loss_kernel = forest_loss_kernel.multiply(forest_loss_kernel.gte(FILTER_LOW_DEFORESTATION))
deforest_kernel = deforest_kernel.multiply(deforest_kernel.gte(FILTER_LOW_DEFORESTATION))
deforest_carbon_kernel = deforest_carbon_kernel.multiply(deforest_kernel.gte(FILTER_LOW_DEFORESTATION))

# avoid divide by zero errors and cap at 1ha/ha
tree_loss_by_human_lu = forest_loss_kernel.divide(non_nat_kernel.add(0.000001)).min(1).max(0)
deforest_by_human_lu = deforest_kernel.divide(non_nat_kernel.add(0.000001))
excess_deforest_per_ha_human_lu = deforest_by_human_lu.where(deforest_by_human_lu.lt(1), 1)
deforest_by_human_lu = deforest_by_human_lu.min(1)
deforest_carbon_by_human_lu = deforest_carbon_kernel.divide(non_nat_kernel.add(0.000001)).divide(excess_deforest_per_ha_human_lu)

forest_loss_kernel = forest_loss_kernel.reproject(PROJECTION, None, SCALE)
deforest_kernel = deforest_kernel.reproject(PROJECTION, None, SCALE)
deforest_carbon_kernel = deforest_carbon_kernel.reproject(PROJECTION, None, SCALE)
tree_loss_by_human_lu = tree_loss_by_human_lu.reproject(PROJECTION, None, SCALE)
deforest_by_human_lu = deforest_by_human_lu.reproject(PROJECTION, None, SCALE)
deforest_carbon_by_human_lu = deforest_carbon_by_human_lu.reproject(PROJECTION, None, SCALE)

cropland = cropland_2019
cropland_2003 = cropland_2003
cropland_change = cropland.And(cropland_2003.Not())

cropland_tree_loss = cropland.multiply(tree_loss_by_human_lu)
cropland_deforest = cropland.multiply(deforest_by_human_lu)
cropland_carbon = cropland.multiply(deforest_carbon_by_human_lu)

non_nat_tree_loss = non_nat.multiply(tree_loss_by_human_lu)
non_nat_deforest = non_nat.multiply(deforest_by_human_lu)
non_nat_carbon = non_nat.multiply(deforest_carbon_by_human_lu)

non_ag_non_natural = non_nat.multiply(cropland.Not())
non_ag_deforest = non_ag_non_natural.multiply(deforest_by_human_lu)

cropland_dLUC = cropland.multiply(loss_excl_nonnat_nonpridist_nontropfire)
non_ag_dLUC = non_ag_non_natural.multiply(loss_excl_nonnat_nonpridist_nontropfire)

cropland_dLUC_carbon = cropland.multiply(deforest_carbon)
non_ag_dLUC_carbon = non_ag_non_natural.multiply(deforest_carbon)

# Export and visualize


Map = geemap.Map()
Map.addLayer(mask, {"min":0, "max":1, "palette": ["black","white"]}, "mask", False)
Map.addLayer(forest_disturbance, {"min":0, "max":1, "palette": ["black","fc2848"]}, "disturbance", False)
Map.addLayer(burned_area, {"min":0, "max":1, "palette": ["black","gray"]}, "burned_area", False)

Map.addLayer(tree_loss_all, {"min":0, "max":1, "palette": ["black","fc2848"]}, "tree loss", False)
Map.addLayer(loss_excl_nonnat, {"min":0, "max":1, "palette": ["black","fc2848"]}, "tree loss excl. non-natural (plantations)", False)
Map.addLayer(loss_excl_nonnat_nonpridist, {"min":0, "max":1, "palette": ["black","fc2848"]}, "tree loss ... & excl nonprimary disturbance (rotation)", False)
Map.addLayer(loss_excl_nonnat_nonpridist_nontropfire, {"min":0, "max":1, "palette": ["black","fc2848"]}, "tree loss ... & excl. fire in non-tropical intact forest => deforest", False)
Map.addLayer(carbon, {"min":0, "max":200, "palette": ["black","darkgreen","green","yellow"]}, "carbon", False)
Map.addLayer(deforest_carbon, {"min":0, "max":200, "palette": ["black","darkred","orange","yellow"]}, "deforest carbon", False)
Map.addLayer(forest_loss_kernel.select(SHOW_RADII), {"min":0, "max":.2, "palette": ["black","fc2848"]}, "loss Kernel", False)
Map.addLayer(deforest_kernel.select(SHOW_RADII), {"min":0, "max":.2, "palette": ["black","red"]}, "deforest Kernel", False)
Map.addLayer(deforest_carbon_kernel.select(SHOW_RADII), {"min":0, "max":50, "palette": ["black","darkred","orange","yellow"]}, "deforest carbon kernel", False)
Map.addLayer(non_nat, {"min":0, "max":1, "palette": ["black","brown"]}, "human landuse excl built", False)
Map.addLayer(non_nat_kernel.select(SHOW_RADII), {"min":0, "max":1, "palette": ["black","brown"]}, "human landuse kernel", False)
Map.addLayer(tree_loss_by_human_lu.select(SHOW_RADII), {"min":0, "max":.5, "palette": ["black","red"]}, "tree loss by human lu", False)
Map.addLayer(deforest_by_human_lu.select(SHOW_RADII), {"min":0, "max":.5, "palette": ["black","red"]}, "deforest risk by human lu", False)
Map.addLayer(deforest_carbon_by_human_lu.select(SHOW_RADII), {"min":0, "max":100, "palette": ["black","red"]}, "deforest carbon risk by human lu", False)


# Cropland related layers

Map.addLayer(cropland.reproject(PROJECTION, None, SCALE), {"min":0, "max": 1, "palette": ["black","teal"]}, "cropland", False)
Map.addLayer(cropland_change.reproject(PROJECTION, None, SCALE), {"min":0, "max": 1, "palette": ["black","cyan"]}, "cropland_change", False)
Map.addLayer(cropland_tree_loss.select(SHOW_RADII).reproject(PROJECTION, None, SCALE), {"min":0, "max": 0.5, "palette": ["black","orange"]}, "cropland tree loss", False)
Map.addLayer(cropland_deforest.select(SHOW_RADII).reproject(PROJECTION, None, SCALE), {"min":0, "max": 0.5, "palette": ["black","orange"]}, "cropland deforest risk", False)
Map.addLayer(cropland_carbon.select(SHOW_RADII).reproject(PROJECTION, None, SCALE), {"min":0, "max": 100, "palette": ["black","darkred","orange","yellow"]}, "cropland deforest carbon", False)
Map.addLayer(non_nat_tree_loss.select(SHOW_RADII).reproject(PROJECTION, None, SCALE), {"min":0, "max": 0.5, "palette": ["black","orange"]}, "non natural tree loss", False)
Map.addLayer(non_nat_deforest.select(SHOW_RADII).reproject(PROJECTION, None, SCALE), {"min":0, "max": 0.5, "palette": ["black","orange"]}, "non natural deforest risk", False)
Map.addLayer(non_nat_carbon.select(SHOW_RADII).reproject(PROJECTION, None, SCALE), {"min":0, "max": 100, "palette": ["black","darkred","orange","yellow"]}, "non natural deforest carbon", False)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
export = {
  #f'deforest_by_human_lu_{2020+HANSEN_START_YEAR}_{SCALE}m':deforest_by_human_lu.select(SHOW_RADII),
  #f'carbon_by_human_lu_{2020+HANSEN_START_YEAR}_{SCALE}m':deforest_carbon_by_human_lu.select(SHOW_RADII),
  #f'cropland_deforest_{2020+HANSEN_START_YEAR}_{SCALE}m':cropland_deforest.select(SHOW_RADII),
  f'non_nat_deforest_{2020+HANSEN_START_YEAR}_{SCALE}m':non_nat_deforest.select(SHOW_RADII),
  #f'cropland_deforest_carbon_{2020+HANSEN_START_YEAR}_{SCALE}m':cropland_carbon.select(SHOW_RADII),
  f'non_nat_deforest_carbon_{2020+HANSEN_START_YEAR}_{SCALE}m':non_nat_carbon.select(SHOW_RADII),
}
for key, image in export.items():
  image = image.rename(key)
  asset_id = str(Path(WORKING_FOLDER, key))
  image = eeUtil.findOrSaveImage(image, asset_id, region=WORLD_GEOM)

In [ ]:
for key, image in export.items():
  asset_id = str(Path(WORKING_FOLDER, key))
  print(asset_id)
  eeUtil.export(asset_id, bucket='landgriffon-gee-bucket', prefix='deforestation', scale=SCALE)

projects/earthengine-legacy/assets/users/fgassert/deforestation_working/non_nat_deforest_2020_1000m
projects/earthengine-legacy/assets/users/fgassert/deforestation_working/non_nat_deforest_carbon_2020_1000m
